# Create Dataset for API


### Part 1: Creating Database

**Introduction**

In this Jupyter notebook requests are sent to the Marvel Developer API to create the dataset which will be used by the API whose code is in the .py file.
The steps don't strictly follow the order of the directions given on the Moodle page as the authors decided to organize the code differently.


**1.1** Install the necessary libraries to run the code.


In [1]:
# Install required libraries
!pip install hashlib
!pip install pandas
!pip install datetime
!pip install requests
!pip install math

ERROR: Could not find a version that satisfies the requirement hashlib (from versions: 20081119)
ERROR: No matching distribution found for hashlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 8.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


**1.2** Setup the Environment

The code is responsible for setting up the environment for interacting with the Marvel API. It begins by importing the necessary libraries. Afterward, it defines the public and private keys needed to access the Marvel API, specifies the base URL and API endpoint, and captures the current timestamp as recommended in the Marvel Developer Portal. In summary, the code initializes the crucial components required to establish authenticated requests to the Marvel API by generating the essential authentication token. Lastly, a function named "m5digest" is implemented, which is tasked with generating an MD5 hash digest for authentication.


In [2]:
# Import library used
import hashlib
import pandas as pd
import datetime
import requests
import math

# Defining the public and private key from the Marvel Developer Portal
publicKey='---'
privateKey='---'

# Defining the base URL and endpoint for the API
urlMARVEL='http://gateway.marvel.com'
pathCharacters='/v1/public/characters'

# Generating a timestamp string using the current date and time for authentication
ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def m5digest(privateKey, publicKey):
    # Generating an MD5 hash based on a combination of a timestamp, private key, and public key
    m5digest= hashlib.md5((ts+privateKey+publicKey).encode('utf-8'))
    return m5digest.hexdigest()

**1.3** Send a request to the API to get a list of 30 characters

The following code is responsible for executing an HTTP GET request to the Marvel Comics API. Initially, it establishes the URL by specifying the characters' path and formulates a set of query parameters. These parameters include the timestamp (ts), API key (publicKey), hash (generated using the m5digest function), and a limit dictating the number of results to be retrieved. Subsequently, the data received from the Marvel API in response to the request is stored in the "characters" variable.


In [3]:
urlMC=urlMARVEL+pathCharacters # Setting the url for the API request

# Defining parameters (timestamp, public key, hash, and limit) for the API request
params={'ts':ts,
        'apikey':publicKey,
        'hash':m5digest(privateKey,publicKey),
        'limit':30}

# Sending a GET request to the Marvel API with specified parameters
response=requests.get(urlMC, params=params)

# Extracting the 'results' data in the JSON format
characters=response.json()['data']['results']

**1.4** Create a Dataframe with the data from the Marvel API

Now, the following code is employed to generate a list for each character that includes their name, ID, events, series, and comics. Following this, a DataFrame (df) is created utilizing the 'comics_list_per_character' data.


In [4]:
comics_list_per_character=[] # Initializing an empty list
for character in characters:
# Creating a list for each character containing their name, ID, events, series, and comics. .
    comics_list_per_character.append([character['name'],
                                      str(character['id']), # As requested in the exercise the character ID will be saved as a string. The type though will be lost once saving it to the CSV file. In the API it is treated as an integer.
                                      character['events']['available'],
                                      character['series']['available'],
                                      character['comics']['available']])
    # In the exercise it is asked to save None when events, series or comics available are 0. The authors decided to use the value 0 as the information is not missing.
    #comics_list_per_character.append([character['name'],character['id'], character['events']['available'] if character['events']['available'] else None, character['series']['available'] if character['series']['available'] else None, character['comics']['available'] if character['comics']['available'] else None])


# Creating a DataFrame using the 'comics_list_per_character' data
df=pd.DataFrame(comics_list_per_character, columns=['Character Name','Character ID','Total Available Events','Total Available Series','Total Available Comics'])
df.head()

,Character Name,Character ID,Total Available Events,Total Available Series,Total Available Comics
0,3-D Man,1011334,1,3,12
1,A-Bomb (HAS),1017100,0,2,4
2,A.I.M.,1009144,0,36,53
3,Aaron Stack,1010699,0,3,14
4,Abomination (Emil Blonsky),1009146,1,28,58


**1.5** Get the most expensive comic for each character

An empty list is initialized to store Character ID and the maximum comic price for the available comics. Subsequently, requests are sent to the Marvel API to retrieve the prices for all available comics for each character. Finally, a DataFrame 'df_comics' is created, containing columns 'Character ID' and 'Price of the Most Expensive Comic'.


In [5]:
pathCOMICS='/v1/public/comics' # Setting the endpoint for comics
list_comics=[] # Initializing an empty list

for i in range(df.shape[0]): # Iterating on every row
    if df['Total Available Comics'][i]!=0:  # Change with this if None is used pd.notna(df['Total Available Comics'][i]):
        k=math.ceil(df['Total Available Comics'][i]/100) # Calculating the number of iterations needed as API only returns up to 100 comics per request.
        list_comic_prices=[] # Initializing an empty list
        for m in range(k):
        # Asking the API k times to retrieve all the data
            params={'ts':ts,
                    'apikey':publicKey,
                    'hash':m5digest(privateKey,publicKey),
                    'characters':df['Character ID'][i],
                    'limit':100,
                    'offset':m*100}
            response=requests.get(url=urlMARVEL+pathCOMICS, params=params).json() # Getting comic data from the API
            for comic in response['data']['results']:
            # Appending comic prices to the list
                for comic_price in comic['prices']:
                    list_comic_prices.append(comic_price['price'])
        list_comics.append([df['Character ID'][i], max(list_comic_prices)]) # Storing Character ID and max comic price
    else:
        list_comics.append([df['Character ID'][i], None]) # Store Character ID and None if no comics are available as also the price is missing and not 0.

# Creating a DataFrame 'df_comics' with columns 'Character ID' and 'Price of the Most Expensive Comic'
df_comics=pd.DataFrame(list_comics,columns=['Character ID','Price of the Most Expensive Comic'])
df_comics.head()

,Character ID,Price of the Most Expensive Comic
0,1011334,2.99
1,1017100,2.99
2,1009144,125.00
3,1010699,2.99
4,1009146,75.00


In [6]:
# Left join of df and df_comics dataframes on the 'Character ID' column
df = pd.merge(df, df_comics, on='Character ID', how='left')
df.head(30)

,Character Name,Character ID,Total Available Events,Total Available Series,Total Available Comics,Price of the Most Expensive Comic
0,3-D Man,1011334,1,3,12,2.99
1,A-Bomb (HAS),1017100,0,2,4,2.99
2,A.I.M.,1009144,0,36,53,125.00
3,Aaron Stack,1010699,0,3,14,2.99
4,Abomination (Emil Blonsky),1009146,1,28,58,75.00
5,Abomination (Ultimate),1016823,0,2,2,3.99
6,Absorbing Man,1009148,5,50,99,99.99
7,Abyss,1009149,1,3,8,9.99
8,Abyss (Age of Apocalypse),1010903,1,3,3,9.99
9,Adam Destine,1011266,0,0,0,NaN


**1.6** Save Dataframe to CSV file

The DataFrame (df) is saved to a CSV file using the .to_csv function with the name 'data.csv'. The "index = False" parameter is employed to exclude the index from being saved in the CSV file since it is not essential information.


In [7]:
df.to_csv('data.csv', index=False)